<a href="https://colab.research.google.com/github/ArshiAbolghasemi/AI-UT/blob/main/knapsack_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knapsack Problem
In this project we're going to solve our knapsack problem using **Genetic Algorithm**

## Problem Description

Consider that we have a knapsack, and we want to go on a picnic. We have some snacks that we want to take with us. Each snack has a value, and we also have a certain amount of each snack that we can take as much as we want. It's obvious that if we take half of a snack, we will have half its value. Also there is some constraints:
- The sum of the values of our snacks shouldn't exceed a special amount that is given to us.
- The number of snacks that we pick should be within the interval that is given to us.
- The amount of the chosen snack shouldn't exceed the available amount of it.

The snack data is located in snacks.csv in the current directory.

# Settings And Imports

In [20]:
#imports
from typing import List
import os
import csv
import random
from functools import reduce

#settings
MAXIMUM_SNACKS_WEIGTHS=10
MINIMUM_SNACKS_VALUES=12
MINIMUM_SNACKS_COUNT=2
MAXIMUM_SNACKS_COUNT=4

INITIAL_POPULATION_SIZE=200

# Inputs
First, lets read input datas from snacks.csv file

In [25]:
CURRENT_DIR = os.getcwd()
SNACKS_CSV_FILE_PATH = os.path.join(CURRENT_DIR, 'snacks.csv')

snacks = []
snacks_available_weight = {}
snacks_values = {}
with open(SNACKS_CSV_FILE_PATH, mode='r') as snacks_file:
    snakcs_csv_reader = csv.reader(snacks_file)

    next(snakcs_csv_reader)

    for row in snakcs_csv_reader:
      snack = row[0]
      snacks.append(snack)
      snacks_available_weight[snack] = row[1]
      snacks_values[snack] = row[2]

# Gene And Chromosme
First, we need a definition for our gene and then using that creating a chromosome, that is a potential solution to our problem.

## Gene
In the context of this problem, each gene represents an item and the chosen amount. If we don't choose an item, its value is equal to zero; otherwise, it is equal to the amount chosen.

In [32]:
class Gene:

  def __init__(self, _snack_name: str, weight: float):
    self.snack_name = _snack_name
    self.weight = weight

  def get_weight(self) -> float:
    return self.weight

  def set_weight(self, _weight) -> None:
    self.weight = _weight;

  def get_value(self) -> float:
    return (
        (self.weight / snacks_available_weight[self.snack_name]) *
        snacks_values[self.snack_name]
    )

  def get_snack_name(self) -> str:
    return self.snack_name

## Chromosome
A chromosome is a collection of genes that represents a potential solution to the knapsack problem. It encodes a set of items to be included in the knapsack.

In [28]:
class Chromosome:

  def __init__(self, _genes: List[Gene]):
    self.genes = _genes

  def get_gense(self) -> List[Gene]:
    return self.genes

  def get_total_weight(self) -> float:
    return reduce(lambda g1, g2: g1.get_weight() + g2.get_weight(), self.genes)

  def get_total_value(self) -> float:
    return reduce(lambda g1, g2: g1.get_value() + g2.get_value(), self.genes)

  def get_snacks_count(self) -> int:
    return len(filter(lambda gene: gene.get_weight() > 0, self.genes))

# Generating Initial Population

In this step, we should generate a population of chromosomes stochastically. Counts of our populatin is defined in our [settings](#scrollTo=k6DVDoLt4kDT)

In [19]:
population = list()

for _ in range(INITIAL_POPULATION_SIZE):
  genes = [
    Gene(snack, random.uniform(0, float(snacks_available_weight[snack])))
    for snack in snacks
  ]
  population.append(Chromosome(genes))

# Fitness Function

Now, we need a fitness function to identify the better chromosome that models our problem. We define the fitness function as follows:

- If total value chromosome is less than minimum snack value or total weight chromosome is more than maximum sancks weight or snacks count is not snacks count interval, fitness is zero
- Otherwise, we consider the fitness score to default to one and then penalize it whenever it breaks constraints.

In [27]:
def fitness(chromosome: Chromosome) -> float:
  total_weight = chromosome.get_total_weight();
  total_value = chromosome.get_total_value();
  snacks_count = chromosome.get_snacks_count();

  if (
      total_weight > float(MAXIMUM_SNACKS_WEIGTHS) or
      total_value < float(MINIMUM_SNACKS_VALUES) or
      not MINIMUM_SNACKS_COUNT < snacks_count < MAXIMUM_SNACKS_COUNT
      ):
     return 0.0

  fitness_score = 1.0
  if total_weight > float(MAXIMUM_SNACKS_WEIGTHS):
    fitness_score -= (
        (total_weight - MAXIMUM_SNACKS_WEIGTHS) /
        total_weight
        )

  if total_value < float(MINIMUM_SNACKS_VALUES):
    fitness_score -= (
        (MINIMUM_SNACKS_VALUES - total_value) /
        MINIMUM_SNACKS_VALUES
        )

  if not MINIMUM_SNACKS_COUNT < snacks_count < MAXIMUM_SNACKS_COUNT:
    fitness_score -= (
        min(abs(snacks_count - MINIMUM_SNACKS_COUNT),
            abs(snacks_count - MAXIMUM_SNACKS_COUNT)) /
             (MAXIMUM_SNACKS_COUNT - MINIMUM_SNACKS_COUNT)
    )

  return max(fitness_score, 0.0)

Fitness one means that we found a solution for the problem

# Crossover and Mutation
Selected chromosomes are used to create offspring through genetic operators such as crossover and mutation. Crossover involves exchanging genetic material between parent chromosomes to create new offspring chromosomes, while mutation introduces small random changes to the chromosomes to maintain diversity in the population.

## Crossover

In [29]:
def crossover(chromosome1: Chromosome, chromosome2: Chromosome) -> Chromosome:
  genes1 = chromosome1.get_genes()
  genes2 = chromosome2.get_genes()
  i = random.randint(0, len(snacks))
  return Chromosome(genes1[:i] + genes[i:])

## Mutation

In [33]:
def mutation(chromosome: Chromosome, mutation_rate: float) -> None:
  genes = chromosome.get_gense()
  for gene in genes:
    if random.choices([True, False], weights=[mutation_rate, 1 - mutation_rate]):
      continue

    if gene.get_weight() > 0.0:
      gene.set_weight(0.0)
    else:
      gene.set_weight(
          random.uniform(
              0,
              float(snacks_available_weight[gene.get_snack_name()])
          )
      )